# Import Libraries

In [2]:
# !pip install google-cloud
# !pip install --upgrade pydata-google-auth google-cloud-bigquery[pandas]
# !pip install google-cloud-storage
# !pip install openpyl


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: C:\Users\RClar\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



Usage:   
  pip <command> [options]

Commands:
  install                     Install packages.
  download                    Download packages.
  uninstall                   Uninstall packages.
  freeze                      Output installed packages in requirements format.
  inspect                     Inspect the python environment.
  list                        List installed packages.
  show                        Show information about installed packages.
  check                       Verify installed packages have compatible dependencies.
  config                      Manage local and global configuration.
  search                      Search PyPI for packages.
  cache                       Inspect and manage pip's wheel cache.
  index                       Inspect information available from package indexes.
  wheel                       Build wheels from your requirements.
  hash                        Compute hashes of package archives.
  completion                  A helper co

ERROR: Could not find a version that satisfies the requirement pandas_gbqinstall (from versions: none)
ERROR: No matching distribution found for pandas_gbqinstall

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: C:\Users\RClar\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: C:\Users\RClar\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: C:\Users\RClar\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [2]:
import re
import os
import numpy as np
import pandas as pd
import openpyxl
import pydata_google_auth
from google.cloud import bigquery
import pydata_google_auth
credentials = pydata_google_auth.get_user_credentials(
    ['https://www.googleapis.com/auth/bigquery'],
)
client = bigquery.Client(project='project-id', credentials=credentials)

In [4]:
# Import Data
query = '''
    Select * 
From `project-id.Boxing_Champs.All_Belt_Data`
'''
All_Belt_Data = client.query(query).to_dataframe()

In [21]:
All_Belt_Data.head(1)

,Belt,Division,Division_Group,No_,Name,Country,Date,Location,Defenses,Days,Ended,Notes,Next_Champ_Date,Day_until_Next_Champ,Region,Continent,Title_Type
0,WBA,Super Welterweight,Super Welterweight,46,Yuri Foreman,Israel,2009-11-14,None,0,203,None,None,2010-06-05,203,Middle East,Asia,None


#  Pre-Process Data 

In [5]:
# To get get an accurate aggregation of the unique fighter in each country, the name is extracted from the Name column


# Define a regex pattern to match the first instance of '(' , '-' , 'def' , or '–'
pattern = r"(.*?)(?:\(|\-|def|–)"

# Extract the first full name from each string
All_Belt_Data['Full_Name'] = All_Belt_Data['Name'].apply(lambda x: re.search(pattern, x).group(1) if re.search(pattern, x) else x)

#Remove whitespace
All_Belt_Data['Full_Name'] = All_Belt_Data['Full_Name'].str.strip()

# Convert datatypes of Days column from string to integer
All_Belt_Data['Days'] = pd.to_numeric(All_Belt_Data['Days'], errors='coerce').convert_dtypes()

# Export to csv to query from BigQuery
All_Belt_Data.to_csv('All_Belt_Data_Cleaned.csv', index=False)

In [6]:
# This dictionary is for standardizing a column with countries 
# that championship bouts took place in, which we will extract later.

location_convert = {
    'Alabama': 'US',
    'Alaska': 'US',
    'Arizona': 'US',
    'Arkansas': 'US',
    'California': 'US',
    'Colorado': 'US',
    'Connecticut': 'US',
    'Delaware': 'US',
    'Florida': 'US',
    'Georgia': 'US',
    'Hawaii': 'US',
    'Idaho': 'US',
    'Illinois': 'US',
    'Indiana': 'US',
    'Iowa': 'US',
    'Kansas': 'US',
    'Kentucky': 'US',
    'Louisiana': 'US',
    'Maine': 'US',
    'Maryland': 'US',
    'Massachusetts': 'US',
    'Michigan': 'US',
    'Minnesota': 'US',
    'Mississippi': 'US',
    'Missouri': 'US',
    'Montana': 'US',
    'Nebraska': 'US',
    'Nevada': 'US',
    'New Hampshire': 'US',
    'New Jersey': 'US',
    'New Mexico': 'US',
    'New York': 'US',
    'North Carolina': 'US',
    'North Dakota': 'US',
    'Ohio': 'US',
    'Oklahoma': 'US',
    'Oregon': 'US',
    'Pennsylvania': 'US',
    'Rhode Island': 'US',
    'South Carolina': 'US',
    'South Dakota': 'US',
    'Tennessee': 'US',
    'Texas': 'US',
    'Utah': 'US',
    'Vermont': 'US',
    'Virginia': 'US',
    'Washington': 'US',
    'West Virginia': 'US',
    'Wisconsin': 'US',
    'Wyoming': 'US',
    'D.C.': 'US',    
    'U.S.': 'US',
    'United States': 'US',
    'United States of America': 'US',
    'London': 'Great Britain',
    'United Kingdom': 'Great Britain',
    'England': 'Great Britain',
    'N/A': 'None',
    'NA': 'None'
}


In [7]:
# The 'Location' columns has the information we want to extract, it contains the location of the championship bouts.
# The general syntax for each row is that the country is listed after the final comma (i.e. Osaka, Japan),
# except for cases with some US states (i.e. Reno, Nevada)

Fighters_AtHome_Vs_Overseas = All_Belt_Data[['Region','Division_Group','Country','Full_Name','Location']]
Fighters_AtHome_Vs_Overseas['Fight_Country'] = Fighters_AtHome_Vs_Overseas['Location'].str.rsplit(',', 1).str[-1].str.strip(' ()').str.strip()
Fighters_AtHome_Vs_Overseas['Fight_Country'] = Fighters_AtHome_Vs_Overseas['Fight_Country'].str.split("(").str[0].str.strip()

C:\Users\RClar\AppData\Local\Temp\ipykernel_560\1352577052.py:6: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  Fighters_AtHome_Vs_Overseas['Fight_Country'] = Fighters_AtHome_Vs_Overseas['Location'].str.rsplit(',', 1).str[-1].str.strip(' ()').str.strip()
C:\Users\RClar\AppData\Local\Temp\ipykernel_560\1352577052.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Fighters_AtHome_Vs_Overseas['Fight_Country'] = Fighters_AtHome_Vs_Overseas['Location'].str.rsplit(',', 1).str[-1].str.strip(' ()').str.strip()
C:\Users\RClar\AppData\Local\Temp\ipykernel_560\1352577052.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [8]:
# After extracting the Location, the column is standardized to so the string matches with the country in the Country column

Fighters_AtHome_Vs_Overseas['Fight_Country'] = Fighters_AtHome_Vs_Overseas['Fight_Country'].apply(lambda x: location_convert.get(x, x))

C:\Users\RClar\AppData\Local\Temp\ipykernel_560\3513717708.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Fighters_AtHome_Vs_Overseas['Fight_Country'] = Fighters_AtHome_Vs_Overseas['Fight_Country'].apply(lambda x: location_convert.get(x, x))


In [9]:
#Function that matches Country with the Fight_Country column to produce Home and Away values as bit values

def match_string(row):
    if row['Country'] == row['Fight_Country']:
        return 1
    else:
        return 0
    
def non_match_string(row):
    if row['Country'] != row['Fight_Country']:
        return 1
    else:
        return 0    

In [10]:
#Match columns and drop null values

Fighters_AtHome_Vs_Overseas['At_Home'] = Fighters_AtHome_Vs_Overseas.apply(match_string, axis=1)
Fighters_AtHome_Vs_Overseas = Fighters_AtHome_Vs_Overseas.dropna(subset=['Country'])
Fighters_AtHome_Vs_Overseas = Fighters_AtHome_Vs_Overseas.dropna(subset=['Fight_Country'])
Fighters_AtHome_Vs_Overseas = Fighters_AtHome_Vs_Overseas[Fighters_AtHome_Vs_Overseas['Fight_Country'] != 'None']

C:\Users\RClar\AppData\Local\Temp\ipykernel_560\3643531998.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Fighters_AtHome_Vs_Overseas['At_Home'] = Fighters_AtHome_Vs_Overseas.apply(match_string, axis=1)


In [160]:
# Impport data from the 'Boxing Champs By Day Table'
champs_by_days = pd.read_csv('Boxing Champs by Day.csv')

In [12]:
#Function to map regions from original dataset to country

def map_region(country):
    return country_dict.get(country, 'Unknown')

In [167]:
champs_by_days.head(1)

,Belt,Continent,Country,Date,Date End,Day,Division,Name,Region,Title Type,Day until Next Champ,Defenses,Full_Name
0,WBA,Asia,Indonesia,11/1/2003,6/27/2009,6/16/2009,Featherweight,Chris John– Regular champion (interim champ p...,Oceania,Regular champion (interim champ promoted),2065,11,Chris John


In [13]:
# Add country and region values to a variable, then convert to dictionary

country_df = All_Belt_Data[['Country','Region']]
country_df = country_df.drop_duplicates(subset='Country', keep='first')
country_dict = country_df.set_index('Country').to_dict()['Region']

# Map Region to countries in a new column

Fighters_AtHome_Vs_Overseas['Region'] = Fighters_AtHome_Vs_Overseas['Country'].apply(map_region)

In [227]:
#Some names have the country in front of the name, this function replaces the countries with a whitespace

def remove_country(name):
    for country in countries:
        if country in name:
            name = name.replace(country, "").strip()
    return name

In [228]:
# Apply function and remove whitespace

countries = ['China', 'Japan', 'Mexico', 'South Korea','Thailand']
champs_by_days['Full_Name'] = champs_by_days['Name'].apply(lambda x: x.split("(")[0].split("def")[0].split("–")[0])
champs_by_days['Full_Name'] = champs_by_days['Full_Name'].apply(remove_country)
champs_by_days['Full_Name'] = champs_by_days['Full_Name'].str.strip()

NameError: name 'champs_by_days' is not defined

In [214]:
# Export Data
champs_by_days.to_csv('Champs_By_Days.csv')

# Query Data From BigQuery

In [113]:
Avg_Days_Per_Beltq = '''
    SELECT 
    Belt, AVG(Days) as Avg_Days
    FROM 
    `project-id.Boxing_Champs.All_Belt_Data_Cleaned` 
    WHERE
    Days != 0
    GROUP BY 
    Belt
    ORDER BY
    Avg_Days DESC
'''
Avg_Days_Per_Belt = client.query(Avg_Days_Per_Beltq).to_dataframe()

In [112]:
#Averages the days per belt
# use in chart to show top value for each belt

Avg_Days_Per_Belt_Daysq = '''
    with defense_days as
    (SELECT DISTINCT
        Region,
        Date,
        Date_End,
        Belt,
        Division,
        Full_Name,
        EXTRACT(YEAR FROM Date) as Year
      FROM 
        `project-id.Boxing_Champs.Champs_By_Days`
      WHERE
        Defenses != 0
      ),

    date_diff as (SELECT DISTINCT 
      Region,
      Belt,
      Division,
      date,
      Date_End,
      Full_Name,
      DATE_DIFF(date_end,date,  DAY) as days
    FROM 
      defense_days 
    Order by 
      date ASC )

    SELECT 
    Region, Belt, CEIL(AVG(days)) as Avg_Days
    FROM 
    date_diff
    WHERE
    Days != 0
    GROUP BY 
    Region,Belt
    ORDER BY
    Belt Asc,Avg_Days DESC

'''
Avg_Days_Per_Belt_Days = client.query(Avg_Days_Per_Belt_Daysq).to_dataframe()

In [135]:
# create chart to get division with top avg 
Avg_Days_Per_Division_Daysq = '''
    with days as
    (SELECT DISTINCT
        Region,
        Date,
        Date_End,
        Belt,
        Division,
        Full_Name,
        EXTRACT(YEAR FROM Date) as Year
      FROM 
        `project-id.Boxing_Champs.Champs_By_Days`

      ),

    date_diff as (SELECT DISTINCT 
      Region,
      Belt,
      Division,
      date,
      Date_End,
      Full_Name,
      DATE_DIFF(date_end,date,  DAY) as days
    FROM 
      days 
    Order by 
      date ASC )

    SELECT 
    Region, Belt, Division, CEIL(AVG(days)) as Avg_Days
    FROM 
    date_diff
    WHERE
    Days != 0
    GROUP BY 
    Region,Belt,Division
    ORDER BY
    Division Asc,Avg_Days DESC
'''
Avg_Days_Per_Division_Days = client.query(Avg_Days_Per_Division_Daysq).to_dataframe()

In [60]:
#Returns the belt that is held on average the longest per country (combine old into new on country)

Belt_country_held_longestq = '''
    with belt_counts as (
      SELECT 
        Region,
        Country,
        Belt,
        CEIL(AVG(Days)) as Avg_Days
      FROM 
        `project-id.Boxing_Champs.All_Belt_Data_Cleaned`
     Where Days != 0
      GROUP BY
        Region,
        Country,
        Belt
      ORDER BY
        Country ASC, Avg_Days DESC
    )

    SELECT 
      Region,
      Country,
      Belt,
      Avg_Days

    FROM (
      SELECT 
        Region,
        Country,
        Belt,
        Avg_Days,
        ROW_NUMBER() OVER (PARTITION BY Country ORDER BY Avg_Days DESC) AS rank
      FROM belt_counts
    )
    WHERE rank = 1
    ORDER BY Avg_Days DESC;
'''
Belt_country_held_longest = client.query(Belt_country_held_longestq).to_dataframe()

In [54]:
Belt_country_held_longest_daysq = '''
    with defense_days as
    (SELECT DISTINCT
        Region,
        Country,
        DATE_DIFF(date_end,date,  DAY) as days,
        Belt
      FROM 
        `project-id.Boxing_Champs.Champs_By_Days`
      WHERE
        Defenses != 0
      ),

    belt_avg as
    (SELECT  
      Region,
      Country,
      Belt,
      CEIL(AVG(days) )as Avg_Days
    FROM 
      defense_days
    Group by
      Region,
      Country,
      Belt
    ) 
    SELECT 
      Region, 
      Country, 
      Belt, 
      Avg_Days
    FROM 
      (
      SELECT 
        Region,
        Country,
        Belt,
        Avg_Days,
        ROW_NUMBER() OVER (PARTITION BY Country ORDER BY Avg_Days DESC) AS rank
      FROM belt_avg
    )
    WHERE rank = 1
    ORDER BY Country DESC;
'''
Belt_country_held_longest_days = client.query(Belt_country_held_longest_daysq).to_dataframe()

In [76]:
# Concatenates the the values from the latter dataframe to the former, that are not already in the former.

Belt_country_held_longest_days.sort_values('Country')
Belt_Longest_Merged = pd.concat([Belt_country_held_longest_days, Belt_country_held_longest[~Belt_country_held_longest.isin(Belt_country_held_longest_days).all(1)]]).drop_duplicates().reset_index(drop=True)
Belt_Longest_Merged=Belt_Longest_Merged.sort_values('Country')

,Region,Country,Belt,Avg_Days
51,South America,Argentina,WBC,723.0
50,Australia,Australia,WBC,760.0
49,Caribbean,Bahamas,WBC,218.0
48,Eastern Europe,Belarus,WBA,413.0
47,South America,Brazil,WBO,1673.0
46,North America,Canada,WBC,841.0
45,East Asia,China,WBC,438.0
44,South America,Colombia,WBO,994.0
43,Eastern Europe,Croatia,WBA,526.0
42,Caribbean,Cuba,WBC,1693.0


In [44]:
#Returns the number of champions from each country

Champions_Per_Countryq = '''
    SELECT 
    Region,Country, COUNT(DISTINCT Full_Name) AS Number_of_Unique_Champions
    FROM 
    `project-id.Boxing_Champs.All_Belt_Data_Cleaned` 
    GROUP BY 
     Region,Country
     ORDER BY
     Number_of_Unique_Champions DESC
'''
Champions_Per_Country = client.query(Champions_Per_Countryq).to_dataframe()

In [42]:
Champions_Per_Country_WBOq = '''
    with champs as (
      SELECT DISTINCT
        Region,
        Country,
        Full_Name
      FROM
      `project-id.Boxing_Champs.Champs_By_Days`
      Where Belt = 'WBO'
    )
    SELECT
     Region,Country, COUNT(DISTINCT Full_Name) AS Number_of_Unique_Champions
    FROM
     champs
    GROUP BY
     Region,Country
     ORDER BY
     Number_of_Unique_Champions DESC
'''
Champions_Per_Country_WBO = client.query(Champions_Per_Country_WBOq).to_dataframe()

In [125]:

# Merge the two dataframes by the 'Country' column
merged_df = pd.merge(Champions_Per_Country, Champions_Per_Country_WBO, on='Country')

# Sum the values in the 'Champions' column where the country names match
merged_df['Number_of_Unique_Champions_x'] = merged_df['Number_of_Unique_Champions_x'] + merged_df['Number_of_Unique_Champions_y']

# Drop the second 'Champions' column
merged_df = merged_df.drop(['Number_of_Unique_Champions_y'], axis=1)

# Rename the first 'Champions' column
merged_df.rename(columns={'Number_of_Unique_Champions_x': 'Champions','Region_x':'Region'}, inplace=True)

# Shape final dataframe
Champions_Per_Country_Final = merged_df[['Region','Country','Champions']]



In [47]:
# Returns count of champions every year for each region.

Champs_Region_Per_Yearq = '''
    WITH champions_per_year AS (
      SELECT 
        Region, 
        EXTRACT(YEAR FROM Date) as Year, 
        COUNT(DISTINCT Full_Name) as Champion_Count
      FROM  `project-id.Boxing_Champs.All_Belt_Data_Cleaned` 
      GROUP BY Region, Year
    )
    SELECT 
      Region, 
      Year, 
      Champion_Count
    FROM champions_per_year
    ORDER BY Year, Champion_Count DESC
'''
Champs_Region_Per_Year = client.query(Champs_Region_Per_Yearq).to_dataframe()

In [202]:
#Returns the division that held the title the longest on average per belt

Longest_Division_Per_Belt_q = '''
      SELECT 
      Region,
      Belt, 
      Division_Group as Division, 
      Full_Name, 
      Days
      FROM `project-id.Boxing_Champs.All_Belt_Data_Cleaned` 
      WHERE Days != 0
      ORDER BY Days DESC
'''
Longest_Division_Per_Belt = client.query(Longest_Division_Per_Belt_q).to_dataframe()

In [167]:
Longest_Division_Per_Belt_Daysq = '''
    with defense_days as
    (SELECT DISTINCT
        Region,
        Division,
        Full_Name,
        DATE_DIFF(date_end,date,  DAY) as days,
        Belt
      FROM 
        `project-id.Boxing_Champs.Champs_By_Days`
      WHERE
        Defenses != 0
      )

    SELECT  
      Region,
      Belt,
      Division,
      Full_Name,
      CEIL(AVG(days) )as Days
    FROM 
      defense_days
    Group by
      Region,
      Belt,
      Division,
      Full_Name

    Order by Days DESC 
'''
Longest_Division_Per_Belt_Days = client.query(Longest_Division_Per_Belt_Daysq).to_dataframe()

In [203]:
# Add the division and standardized division_group to a dictionary then map to new data
division_dict = All_Belt_Data.set_index('Division')['Division_Group'].to_dict()
Longest_Division_Per_Belt_New['Division'] = Longest_Division_Per_Belt_New['Division'].map(division_dict)


ldfn = Longest_Division_Per_Belt_New.copy()
ldfo = Longest_Division_Per_Belt.copy()

# Merge the two dataframes
Longest_Division_Final = combined_df = pd.concat([ldfn, ldfo], ignore_index=True)
Longest_Division_Final = Longest_Division_Final.sort_values('Days',ascending=False)
Longest_Division_Final.drop_duplicates(subset=["Division", "Full_Name"], keep="first", inplace=True)

In [51]:
# Return the champion with the most total defenses per belt

Top_Defense_Per_Belt_All_Timeq = '''
    with unique_names as(
    SELECT DISTINCT 
      Belt, Full_Name, Defenses
    FROM
     `project-id.Boxing_Champs.Champs_By_Days`
     ORDER BY
      Full_Name DESC
    )

    SELECT
      Belt,
      Full_Name,
      Total_Defenses

    FROM (
      SELECT 
        Belt,
        Full_Name,
        Total_Defenses,
        ROW_NUMBER() OVER (PARTITION BY Belt ORDER BY Total_Defenses DESC) AS rank
      FROM (
        SELECT
          Belt,
          Full_Name,
          SUM(Defenses) AS Total_Defenses
        FROM 
          unique_names 
        GROUP BY 
          Belt,
          Full_Name 


      )
    )
    WHERE rank = 1
    ORDER BY Total_Defenses DESC;
'''
Top_Defense_Per_Belt_All_Time = client.query(Top_Defense_Per_Belt_All_Timeq).to_dataframe()

In [53]:
Yearly_Total_Defenses_Per_Beltq = '''
    with yearly_defenses as
    (SELECT DISTINCT
        Region,
        Country,
        Belt,
        EXTRACT(YEAR FROM Date) as Year, 
        Defenses
      FROM 
        `project-id.Boxing_Champs.Champs_By_Days`
      Where
        Defenses != 0   
    )

    SELECT
    Region,
      Country,
      Belt,
      Year,
      SUM(Defenses) as Total_Defenses
    FROM
    yearly_defenses
    GROUP BY
        Region,
        Country,
        Belt,
        Year
    ORDER BY
        Belt ASC, Year ASC
    
'''
Yearly_Total_Defenses_Per_Belt = client.query(Yearly_Total_Defenses_Per_Beltq).to_dataframe()

# Export Data to Tableau

In [21]:
# Export to xlsx to for Tableau

def dftoXlsx(df,name):
    xlsx = './%s.xlsx' % name
    df.to_excel(xlsx, sheet_name=name, index=False)

In [145]:
# import use in chart to show top value for each belt 
Avg_Days_Per_Belt_export = dftoXlsx(Avg_Days_Per_Belt_New,'Avg_Days_Per_Belt')

In [146]:
#import
Avg_Days_Per_Division_export = dftoXlsx(Avg_Days_Per_Division_New,'Avg_Days_Per_Division')

In [133]:
# import use in chart to show top value for each belt 
Belt_Held_Longest_export = dftoXlsx(Belt_Longest_Merged,'Belt_Held_Longest')

In [126]:
# import
Champions_Per_Country_export = dftoXlsx(Champions_Per_Country_Final,'Champions_Per_Country_Final')

In [210]:
# import
Champs_Region_Per_Year_export = dftoXlsx(Champs_Region_Per_Year,'Champs_Region_Per_Year')

In [23]:
# import
Fighters_AtHome_Vs_Overseas_export = dftoXlsx(Fighters_AtHome_Vs_Overseas,'Fighters_AtHome_Vs_Overseas')

In [216]:
# import Chart to show the division with the highest avg day per belt
Longest_Division_Per_Belt_export = dftoXlsx(Longest_Division_Final,'Longest_Division_Per_Belt')

In [97]:
# import
Top_Defense_Per_Belt_All_Time_export = dftoXlsx(Top_Defense_Per_Belt_All_Time,'Top_Defense_Per_Belt_All_Time')

In [99]:
# import
Yearly_Total_Defenses_Per_Belt_export = dftoXlsx(Yearly_Total_Defenses_Per_Belt,'Yearly_Total_Defenses_Per_Belt')

In [22]:
# define a function that reads xlsx files and adds it to a workbook


def add_to_workbook(file_names, workbook_name):
    workbook = openpyxl.Workbook()
    
    for file_name in file_names:
        data = openpyxl.load_workbook(file_name)
        sheet = data.active
        new_sheet = workbook.create_sheet(file_name)
        
        for row in sheet.iter_rows():
            for cell in row:
                new_sheet[cell.coordinate] = cell.value
                
    workbook.save(workbook_name)

In [24]:
# list of xlsx files
files = ['Avg_Days_Per_Belt.xlsx', 
         'Avg_Days_Per_Division_a.xlsx', 
         'Belt_Held_Longest.xlsx',         
         'Champions_Per_Country_Final.xlsx', 
         'Champs_Region_Per_Year.xlsx',
         'Fighters_AtHome_Vs_Overseas.xlsx', 
         'Longest_Division_Per_Belt.xlsx', 
         'Top_Defense_Per_Belt_All_Time.xlsx', 
         'Yearly_Total_Defenses_Per_Belt.xlsx']

# name of the workbook to save the data
workbook_name = 'Boxing_Dat.xlsx'

# call the function to add the xlsx files to the workbook
add_to_workbook(files, workbook_name)

C:\Users\RClar\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")
